In [1]:
! pip install openai-agents
! pip install resend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.3/223.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00


# Use OpenAI & ReSend API KEY

In [2]:
from kaggle_secrets import UserSecretsClient
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict

import os
import asyncio
import resend

# Setting Up API Keys for OpenAI and Resend

In [3]:

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("OPENAI_API_KEY")
secret_value_1 = user_secrets.get_secret("RESEND_API_KEY")
resend.api_key = secret_value_1
os.environ["OPENAI_API_KEY"] = secret_value_0

# Sending a Test Email Using Resend

In [4]:
def send_test_email():
    params = {
        "from": "onboarding@resend.dev",
        "to": ["evil@gmail.com"],
        "subject": "Test email",
        "text": "This is an important test email"
    }
    response = resend.Emails.send(params)
    print("Email sent successfully!")
    print(response)

send_test_email()

Email sent successfully!
{'id': 'f1b4c3db-ac2d-49fb-9c4b-3f6764842c76'}


# SecureTech Sales Agent Instructions

In [5]:
instructions1 = "You are a sales agent working for SecureTech, \
a company providing advanced AI-driven endpoint cybersecurity solutions. \
You write professional, serious cold emails to CTOs and IT managers."

instructions2 = "You are a humorous, engaging sales agent working for SecureTech, \
a company providing AI-driven endpoint cybersecurity solutions. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for SecureTech, \
a company providing AI-driven endpoint cybersecurity solutions. \
You write concise, to-the-point cold emails."


# SecureTech Sales Agents

**defined three specialized sales agents, each with a unique style for generating cold sales emails.**

1️⃣ Professional Sales Agent
- Name: Professional Sales Agent
- Purpose: Generates formal and serious cold emails.
- Target Audience: CTOs, IT managers, and other technical decision-makers.
- Style: Professional, trustworthy, and focused on clear value propositions.

2️⃣ Engaging Sales Agent
- Name: Engaging Sales Agent
- Purpose: Creates witty and engaging cold emails.
- Target Audience: Prospects who respond well to humor and creative messaging.
- Style: Humorous, attention-grabbing, and designed to encourage responses.

3️⃣ Busy Sales Agent
- Name: Busy Sales Agent
- Purpose: Produces concise and to-the-point emails.
- Target Audience: Time-sensitive executives and busy decision-makers.
- Style: Short, clear, and efficient, with no unnecessary content.

In [6]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini"
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini"
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini"
)


# Streaming Cold Sales Email Generation

**The following workflow shows how to generate a cold sales email using a sales agent and stream the response as it is being produced:**

In [7]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Enhance Your Cybersecurity Strategy with AI-Driven Solutions

Dear [Recipient's Name],

I hope this message finds you well.

As the cybersecurity landscape becomes increasingly complex, organizations are faced with mounting challenges to protect their sensitive data and infrastructure. At SecureTech, we specialize in providing advanced AI-driven endpoint cybersecurity solutions tailored to meet the unique needs of businesses like yours.

Our innovative technology offers real-time threat detection, streamlined response protocols, and comprehensive visibility into your network. By leveraging the power of artificial intelligence, we empower IT teams to stay ahead of emerging threats, ensuring robust protection for your endpoints without compromising operational efficiency.

I would like to propose a brief conversation to discuss how SecureTech can enhance your cybersecurity strategy and mitigate risks within your organization. Would you be available for a 20-minute call next week

# Generating Cold Sales Emails in Parallel

##### This code demonstrates how to use multiple sales agents at the same time to generate cold sales emails, then collect and display their outputs.

In [8]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Enhance Your Cybersecurity Posture with AI-Driven Solutions

Dear [Recipient's Name],

I hope this message finds you well. As the cyber threat landscape continues to evolve, I wanted to introduce you to SecureTech’s advanced AI-driven endpoint cybersecurity solutions that can help strengthen your organization’s defenses.

Our platform offers comprehensive protection against sophisticated threats, including ransomware and zero-day attacks, ensuring your critical data and systems remain secure. Key features include:

- **Real-time Threat Detection:** Leverage AI to identify and respond to threats before they can cause harm.
- **Automated Incident Response:** Reduce response times with automated actions that mitigate risks effectively.
- **Seamless Integration:** Our solutions easily integrate with your existing infrastructure, ensuring minimal disruption to your operations.

I would love to schedule a brief call to discuss how SecureTech can enhance your organization’s cybersecu

# Selecting the Best Cold Sales Email
> This workflow shows how to generate multiple cold emails and automatically pick the best one using an expert agent.

# Steps Explained

- Define the Cold Email Selector Agent
- Name: Cold Email Selector
- Purpose: Evaluates multiple cold email drafts as if it were the recipient (CTO, IT manager, or decision-maker).

# Instructions:
- Focus on professionalism, clarity, and engagement.
- Do not provide explanations; return only the selected email text.
- Generate multiple emails
- Use three sales agents (Professional, Engaging, Busy) to create different versions of a cold sales email.
- asyncio.gather runs all agents in parallel, speeding up generation.

# Combine the outputs
- All generated emails are combined into a single string for the selector agent.
- This makes it easy for the Cold Email Selector to compare options at once.

# Select the best email

- The selector agent evaluates the combined emails and chooses the one most likely to get a response.
- Only the selected email is returned, without any explanations.
- Display the result
- The best cold sales email is printed for review or further use.

# ✅ Benefits
- Ensures pick the most effective email from multiple styles.
- Saves time by automating the evaluation process.
- Provides a structured workflow for generating, comparing, and sending cold emails.

In [9]:
sales_picker = Agent(name="Cold Email Selector",instructions=(
        "You are an expert sales agent evaluating cold sales emails. "
        "Given multiple email options, imagine you are the recipient (a CTO, IT manager, or decision-maker) "
        "and select the one you are most likely to respond to. "
        "Focus on professionalism, clarity, and engagement. "
        "Do NOT provide explanations or reasoning—reply with the selected email text only."),model="gpt-4o-mini")


message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(Runner.run(sales_agent1, message),Runner.run(sales_agent2, message),Runner.run(sales_agent3, message))
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Strengthen Your Endpoint Security with AI-Driven Solutions

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from SecureTech, where we specialize in advanced AI-driven endpoint cybersecurity solutions designed to protect your organization against emerging threats.

As cyber threats continue to evolve, traditional security measures often fall short in providing comprehensive protection. Our platform leverages cutting-edge machine learning algorithms to detect and respond to potential vulnerabilities in real time, allowing your team to focus on driving innovation rather than constantly managing security risks.

Key benefits of our solution include:

- **Proactive Threat Detection:** Identify and mitigate risks before they impact your operations.
- **Seamless Integration:** Our solutions easily integrate with your existing IT infrastructure, minimizing deployment disruptions.
- **Scalability:** Whether yo

# SecureTech Sales Agents Overview

# 1️⃣ Professional Sales Agent
- Name: Professional Sales Agent
- Purpose: Writes formal and serious cold emails.
- Target Audience: CTOs, IT managers, and other technical decision-makers.
- Style: Professional, trustworthy, and focused on clear value propositions.

# 2️⃣ Engaging Sales Agent
- Name: Engaging Sales Agent
- Purpose: Creates witty and engaging cold emails.
- Target Audience: Prospects who respond well to humor and creative messaging.
- Style: Humorous, attention-grabbing, and designed to encourage responses.

# 3️⃣ Busy Sales Agent
- Name: Busy Sales Agent
- Purpose: Produces concise and to-the-point emails.
- Target Audience: Busy executives and decision-makers with limited time.
- Style: Short, clear, and efficient, avoiding unnecessary content.

In [10]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini",
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini",
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini",
)


# **send_email** Function Using Resend

In [11]:
@function_tool
def send_email(body: str):
    params = {"from": "onboarding@resend.dev","to": ["evil@gmail.com"],"subject": "Sales email","text": body}
    response = resend.Emails.send(params)
    return {"status": "success", "response": response}


send_email

FunctionTool(name='send_email', description='', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fd3f083c7c0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

# Converting a Sales Agent into a Tool

# Steps Explained
- Agent as a Tool
- sales_agent1.as_tool(...) converts the Professional Sales Agent into a tool.
- This allows other agents or workflows to invoke it programmatically without directly interacting with the agent code.
- Tool Name and Description
- tool_name: A unique identifier for the tool (sales_agent1).
- tool_description: Explains the purpose of the tool, e.g., “Write a cold sales email.”
- These parameters help in organizing and documenting tools in your system.
- Printing the Tool
- print(f"Tool No 1: {tool1}") displays the tool’s metadata, confirming it’s successfully created.

# ✅ Benefits
- Enables modular agent workflows, where agents can call tools instead of writing logic themselves.
- Useful for multi-agent systems or orchestrating complex tasks.
- Makes it easier to reuse agents in different parts of a pipeline.

In [12]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
print(f"Tool No 1: {tool1}")

Tool No 1: FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fd3f083cfe0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)


# Creating a Collection of Tools for Sales Email Automation

> This example shows how to wrap multiple sales agents and an email-sending function as tools, making them ready for use in a multi-agent workflow.

## Steps Explained
#### Define a Description
- description = "Write a cold sales email"
- A short, clear description used for all sales agent tools to explain their purpose.
- Convert Sales Agents into Tools
- tool1 = sales_agent1.as_tool(...)
- tool2 = sales_agent2.as_tool(...)
- tool3 = sales_agent3.as_tool(...)
- Each agent is wrapped as a tool with a name and a description, allowing them to be invoked programmatically.

#### Combine Tools into a List

- tools = [tool1, tool2, tool3, send_email]
- Includes the three sales agent tools plus the email-sending function (send_email).
- This list can now be passed to other agents or workflows that need to generate and send emails.

#### ✅ Benefits
- Provides a modular workflow where agents and functions can interact seamlessly.
- Makes it easy to generate multiple email drafts and send them automatically.
- Simplifies management of tools in complex multi-agent systems.

In [13]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fd3f083d760>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fd3f191bc40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', description='

# Automated Sales Manager Workflow

> This workflow demonstrates how a Sales Manager agent can automatically generate, evaluate, and send the most effective cold sales email using multiple specialized sales agents.

## Workflow Overview
## 1️⃣ Generate Drafts
- The Sales Manager calls all three sales agent tools:
- Professional Sales Agent
- Engaging Sales Agent
- Busy Sales Agent
- Each tool produces a unique cold email draft.
- Important: Drafts must come from the tools, not manually written.

## 2️⃣ Evaluate Drafts
- The Sales Manager reviews all three drafts.
- Selection is based on:
- Clarity: Easy to read and understand.
- Engagement: Likely to capture the founder’s attention.
- Persuasiveness: Motivates the recipient to take action.
- Only the best email is chosen for sending.

## 3️⃣ Send the Best Email
- The Sales Manager uses the send_email tool to deliver the selected draft.
- Only one email is sent to avoid spamming or confusion.
### Rules & Constraints
- All drafts must come from the sales agent tools.
- No manual editing of drafts is allowed; the agent only selects the best draft.
- The send_email tool must be used exactly once.

## Implementation Details
- Agent Name: Sales Manager
- Tools Used: sales_agent1, sales_agent2, sales_agent3, send_email
- Target Recipient: Founder (Dear Founder)
- Model: GPT-4o-mini
- Max Turns: 20 (ensures the agent has enough steps to complete the workflow)



In [14]:
instructions = """
You are the Sales Manager at ComplAI. Your goal is to identify and send the single most effective cold sales email to the founder using the three sales_agent tools.

Workflow:
1. Generate Drafts: Call all three sales_agent tools to produce three distinct cold email drafts. Do not skip any tool and do not generate drafts manually.
2. Evaluate Drafts: Review the three drafts critically. Select the one that is most likely to get a positive response from the founder, based on clarity, engagement, and persuasiveness.
3. Send the Best Email: Use the send_email tool to send ONLY the selected draft to the founder. Never send more than one email.

Rules:
- All drafts must come from the sales_agent tools.
- You may not modify the drafts yourself; only select the best one.
- Ensure the send_email tool is used exactly once.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini"
)

message = "Send a cold sales email addressed to 'Dear Founder'"

with trace("Sales Manager Workflow"):
    result = await Runner.run(sales_manager, message,max_turns=20)

# Email Enhancement Agents
## 1️⃣ Subject Line Writer
- Purpose: Create engaging, concise subject lines to maximize open rates.
- Agent: Email Subject Writer (GPT-4o-mini)
- Tool: subject_writer – generates compelling subject lines from email content.

## 2️⃣ HTML Email Body Converter
- Purpose: Convert plain text or markdown emails into clean, readable HTML.
- Agent: HTML Email Body Converter (GPT-4o-mini)
- Tool: html_converter – formats emails for clarity and engagement.

## ✅ Benefits
- Professional, high-converting emails.
- Ready-to-send HTML format.
- Easily integrated into automated workflows.

In [15]:
subject_instructions = """
You are an expert email copywriter. Your task is to craft an engaging, concise, and compelling subject line
for a cold sales email. You are given the main email message and must write a subject that maximizes
open rates and encourages the recipient to read the email.
"""

html_instructions = """
You are an expert email designer. Your task is to convert a plain text or markdown email body
into a clean, well-formatted HTML email. The HTML should be simple, readable, visually appealing,
and optimized for clarity and engagement.
"""

subject_writer = Agent(
    name="Email Subject Writer",
    instructions=subject_instructions,
    model="gpt-4o-mini"
)
subject_tool = subject_writer.as_tool(
    tool_name="subject_writer",
    tool_description="Generate a compelling subject line for a cold sales email"
)

html_converter = Agent(
    name="HTML Email Body Converter",
    instructions=html_instructions,
    model="gpt-4o-mini"
)
html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert a text email body to a well-formatted HTML email"
)


# HTML Email Sending Tool
## Function: send_html_email
- Purpose: Send an email with a subject line and HTML body using Resend.
## Inputs:
- subject – The email subject line.
- html_body – The email content in HTML format.
- Output: Returns a dictionary with the status and Resend response.

## Tool Collection
- subject_tool – Generates a compelling email subject line.
- html_tool – Converts plain text emails to well-formatted HTML.
- send_html_email – Sends the email to the recipient.

## ✅ Benefits
- Automates the process from subject creation to HTML formatting to email sending.
- Ensures professional, engaging, and visually appealing emails.
- Can be easily integrated into an automated sales workflow.

In [16]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    params = {"from": "onboarding@resend.dev","to": ["evil@gmail.com"],"subject": subject,"html": html_body}
    response = resend.Emails.send(params)
    return {"status": "success", "response": response}


tools = [subject_tool, html_tool, send_html_email]

print(f"Tools : {tools}")

Tools : [FunctionTool(name='subject_writer', description='Generate a compelling subject line for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fd3f083d3a0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='html_converter', description='Convert a text email body to a well-formatted HTML email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fd3f3d16700>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None

# Email Manager Agent
## Purpose
- The Email Manager formats and sends professional cold sales emails automatically.

## Workflow
- Generate Subject: Use the subject_writer tool to create a compelling email subject line.
- Convert to HTML: Use the html_converter tool to format the plain text email into a visually appealing HTML email.
- Send Email: Use the send_html_email tool to deliver the email to the recipient.

## Tools
- tool1, tool2, tool3 – Sales agents generating email drafts.
- subject_writer – Generates engaging subject lines.
- html_converter – Converts text emails to HTML.
- send_html_email – Sends the formatted email via Resend.

## Handoffs
- The Email Manager agent receives the selected draft and ensures it is formatted and sent correctly.

## ✅ Benefits
- Fully automates draft selection, formatting, and sending of cold sales emails.
- Ensures emails are professional, engaging, and visually appealing.
- Integrates seamlessly into a multi-agent sales workflow.

In [17]:
instructions = """
You are the Email Manager. Your job is to format and send professional cold sales emails. 
Follow these steps:

1. Use the subject_writer tool to generate a compelling subject line for the email.
2. Use the html_converter tool to convert the plain text email body into a well-formatted HTML email.
3. Use the send_html_email tool to send the email with the generated subject and HTML body.

Ensure the email is clear, engaging, and professional. Follow the tools in order and do not skip any step.
"""

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Format the email into HTML and send it to the recipient"
)


tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fd3f083d760>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fd3f191bc40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent3', description='Wr

In [18]:
sales_manager_instructions = """
You are the Sales Manager at ComplAI. Your objective is to identify and send the single most effective cold sales email using the sales_agent tools.

Workflow:
1. Generate Drafts: Use all three sales_agent tools to produce three distinct email drafts. Do not proceed until all drafts are ready.
2. Evaluate and Select: Critically review the drafts and select the one that is most likely to get a positive response. You may call the tools multiple times if needed to improve results.
3. Handoff for Sending: Provide ONLY the selected email draft to the 'Email Manager' agent. The Email Manager will format it and send it.

Rules:
- All drafts must come from the sales_agent tools — do not write any drafts yourself.
- Only hand off ONE email to the Email Manager — never more than one.
- Ensure the email is clear, professional, and engaging.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

message = "Send a cold sales email addressed to 'Dear CEO' from Alice"

with trace("Automated SDR Workflow"):
    result = await Runner.run(sales_manager, message,max_turns=20)
